<a href="https://colab.research.google.com/github/akikoiwamizu/enron-vizards/blob/main/data/Enron_data_cleansing_collab_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Downloading the file from Kaggle requires you to go to the account tab of the My Profile 
# section and click on Create New API Token. This will download a kaggle.json file.

## ! mkdir /root/.kaggle/
## ! cp kaggle.json /root/.kaggle/
## ! chmod 600 ~/.kaggle/kaggle.json

# Once you have the kaggle.json file download, move it to this location on your local machine
# to access the Kaggle API. For me, the following command did the trick:

##! cp ~/Downloads/kaggle.json  ~/.kaggle/
##! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# This will fail if the files are already on your local machine in this directory.

##! kaggle datasets download wcukierski/enron-email-dataset
##! unzip enron-email-dataset.zip 

In [ ]:
# Remove zip file after csv successfully obtained.

##! rm enron-email-dataset.zip 

# Enron data cleansing
## Import dataset

In [ ]:
import pandas as pd
import numpy as np
import os
from email.parser import Parser
from datetime import datetime
import re

In [ ]:
df = pd.read_csv('emails.csv')
df.head()

In [ ]:
emails_lst = []

for i in range(len(df)):
    email = Parser().parsestr(df.message[i])
    emails_lst.append(dict(zip(email.keys(), email.values())))
    emails_lst[-1]["Body"] = email.get_payload()

In [ ]:
emails = pd.DataFrame(emails_lst)
emails.head()

We just want to keep the useful stuff and drop all unnecessary columns.

In [ ]:
emails.drop(['Message-ID','Mime-Version','Content-Type','Content-Transfer-Encoding','X-FileName'], axis=1, inplace=True)
emails.head()

Now let's have a look on the data.

In [ ]:
display(emails.describe())
emails.isna().sum()

There seem to be a couple of NaNs for the authors and recicpients of some emails which makes them useless for our analysis. We hence drop them.

In [ ]:
emails.dropna(subset=["X-From","X-To"], how="all", inplace=True)
emails.reset_index(drop=True,inplace=True)
emails.isna().sum()

Let's try to get the date into a nice format.

In [ ]:
emails = pd.concat((emails, pd.DataFrame(np.reshape([y for x in emails.Date.apply(lambda x: x.split()[1:6]) for y in x],
                                                (len(emails),5)), 
             columns=["day","month","year","time","tzdiff"])), axis=1)
emails.head()

In [ ]:
#Convert to int
emails.day = emails.day.astype(int)
emails.year = emails.year.astype(int)
#Convert month name to number
d = dict(zip(pd.date_range('2000-01-01', freq='M', periods=12).strftime('%b'),range(1,13)))
emails.month = emails.month.map(d)

The Enron scandal ended with its bankrupcy in Dec 2001. Hence, data before this date should be irrelevant or non-existent. But we could also go for 2007 when the company completely ceased to exist. Let's check:

In [ ]:
display(emails[emails.year > 2001])
display(emails[emails.year > 2007])

There is some erroneous data with years in the future. Let's stick to the 2007 threshold then.

In [ ]:
emails = emails[emails.year <= 2007]
emails.head()

What about minimum values? Enron was founded in 1985 so that seems to be a reasonable threshold. However, Enron was founded by a merger of two companies and one of them was founded in 1931. But emails were invented and used in companies from 1975 on. Let's use this a threshold instead.

In [ ]:
emails[emails.year < 1975]

Seems like we will drop junk data. Let's do that.

In [ ]:
emails = emails[emails.year >= 1975]
emails.head()

Ok, now let's take a look on the names of people.

In [ ]:
people = set().union(*[emails["X-To"],emails["X-From"]])
people

There are some issues with the names. Let's fix them.

In [ ]:
#Let's get rid of everything after "<"
emails["X-To"] = emails["X-To"].apply(lambda x: x[:x.find("<")-1] if x.find("<") > 0 else x)
emails["X-From"] = emails["X-From"].apply(lambda x: x[:x.find("<")-1] if x.find("<") > 0 else x)
emails["X-cc"] = emails["X-cc"].apply(lambda x: x[:x.find("<")-1] if x.find("<") > 0 else x)
emails["X-bcc"] = emails["X-bcc"].apply(lambda x: x[:x.find("<")-1] if x.find("<") > 0 else x)
emails[["X-To","X-From","X-cc","X-bcc"]].head()

In [ ]:
annoying_lst = emails["X-To"][emails["X-To"].str.len().sort_values(ascending=False)[:10].index]
less_annoying_lst = emails["X-To"].sample(20)

In [ ]:
def clean_name_format(names):
    #Variables
    emails_dict = {}
    emails_list = []
    name_drop = []
    remaining = []
    single = []
    multiple = []
    clean_names = []
    #One person with format "lastName, firstName optMiddleName"
    if names.strip(",").count(",") == 1:
        if len(names.split()) > 3:
            multiple = names.strip("\'").split(", ")
        else:
            single = names.strip("\'").split(", ")
            if len(single) == 1:
                single = single[0].strip("\'").split(",")
            single = single[1] + " " + single[0]
    else:
        #Separate string into list of strings
        names = names.strip(",").split(", ")
        #Separate email addresses from name and save in dictionary
        names = [x.replace('\\',"").replace('\"',"").strip(' ') for x in names]
        for name in names:
            #Assuming only one email per name
            email = re.findall(r'<[\w.+-]+@[\w-]+\.[\w.-]+>', name)
            if len(email) > 0:
                if len(name) > len(email[0]):
                    emails_dict[email[0]] = name.replace(email[0],"").replace("."," ").strip(" ").title()
                else:
                    emails_list.append(email[0].replace("."," ").title())
                name_drop.append(name)
        #Lookup names in email_dict
        for i, email in enumerate(emails_list):
            if email in emails_dict.values():
                emails_list[i] = emails_dict.get(i)    
        remaining = list(set(names) - set(name_drop))
        clean_names = list(set(emails_list + list(emails_dict.values()) + remaining))
    if single:
        clean_names.append(single)
    if multiple:
        clean_names = clean_names + multiple
    return clean_names

In [ ]:
#Let's extract lists of recipients
emails["X-To"] = emails["X-To"].apply(lambda x: clean_name_format(x))
emails["X-From"] = emails["X-From"].apply(lambda x: clean_name_format(x))
emails["X-cc"] = emails["X-cc"].apply(lambda x: clean_name_format(x))
emails["X-bcc"] = emails["X-bcc"].apply(lambda x: clean_name_format(x))

In [ ]:
emails.head()

Let's see if this looks better now.

In [ ]:
#people = set().union(*[emails["X-To"].apply(pd.Series).stack().reset_index(drop=True),
#              emails["X-From"].apply(pd.Series).stack().reset_index(drop=True),
#              emails["X-cc"].apply(pd.Series).stack().reset_index(drop=True),
#              emails["X-bcc"].apply(pd.Series).stack().reset_index(drop=True)])
#print(len(people))
#people

Looks a lot better now. The EDA will show if we have other issues with the data, but for now...

In [ ]:
# Email recipients limited to highest 10k
authors = pd.Series([item for row in emails["X-From"] for item in row if item])
authors = pd.DataFrame({"Name" : authors.value_counts().index,
                            "Sent" : authors.value_counts()}).reset_index(drop=True).sort_values(by="Sent", ascending=False)

# Email recipients limited to highest 10k
all_recipients = emails["X-To"] + emails["X-cc"] + emails["X-bcc"]
recipients = pd.Series([item for row in all_recipients for item in row if item])
recipients = pd.DataFrame({"Name" : recipients.value_counts().index,
                            "Received" : recipients.value_counts()}).reset_index(drop=True).sort_values(by="Received", ascending=False)

In [ ]:
# Define culprits and clean dataset for different spellings
key_people = ['Kenneth Lay', 'Jeffrey Skilling', 'Andrew Fastow']

kenneth_lay = authors.Name[authors.Name.apply(lambda x: "Ken Lay" in x)]
jeff_skilling = authors.Name[authors.Name.apply(lambda x: "Jeff Skilling" in x)]
andrew_fastow = ["Andrew S Fastow"]

# Clean dataset with correct names
def clean_names(name_list):
    for i, name in enumerate(name_list):
        if name in kenneth_lay:
            name_list[i] = 'Kenneth Lay'
        elif name in jeff_skilling:
            name_list[i] = 'Jeffrey Skilling'
        elif name in andrew_fastow:
            name_list[i] = 'Andrew Fastow'
        return name_list
    
emails["X-From"] = emails["X-From"].apply(lambda x: clean_names(x))
emails["X-To"] = emails["X-To"].apply(lambda x: clean_names(x)) 
emails["X-cc"] = emails["X-cc"].apply(lambda x: clean_names(x)) 
emails["X-bcc"] = emails["X-bcc"].apply(lambda x: clean_names(x)) 

In [ ]:
emails.to_json("emails_clean.json")